<a href="https://colab.research.google.com/github/tamburins/ESAA_2023/blob/main/%E1%84%8B%E1%85%A1%E1%86%BC%E1%84%89%E1%85%A1%E1%86%BC%E1%84%87%E1%85%B3%E1%86%AF_%E1%84%8B%E1%85%A7%E1%86%AB%E1%84%89%E1%85%B3%E1%86%B8%E1%84%86%E1%85%AE%E1%86%AB%E1%84%8C%E1%85%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [16]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


직접 투표 분류기는 앙상블에 있는 각 분류기의 선택을 카운트해서 가장 많이 카운트된(투표된)클래스를 선택하나, 간접투표 분류기는 평균 확률 추정을 구해서 가장 높은 확률을 가진 클래스를 선택한다.

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

학습률을 감소시켜야 한다. 예측기 수가 너무 많으면 알맞은 개수를 찾기 위해 조기종료 기법을 사용할 수 있다.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [1]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

/usr/local/lib/python3.9/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(X,y,random_state=42)


####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다. 

In [3]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [9]:
# model fitting
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, random_state=42)
etc_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

rnd_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)
etc_clf.fit(X_train, y_train)
mlp_clf.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [7]:
from sklearn.ensemble import VotingClassifier

In [11]:
# model fitting
voting_clf = VotingClassifier(
    estimators=[('mlp', mlp_clf),('rnd', rnd_clf), ('svm', svm_clf), ('etc', etc_clf)],
    voting = 'hard')
voting_clf.fit(X_train, y_train)


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('mlp', MLPClassifier(random_state=42)),
                             ('rnd', RandomForestClassifier(random_state=42)),
                             ('svm', LinearSVC(max_iter=100, random_state=42)),
                             ('etc', ExtraTreesClassifier(random_state=42))])

In [12]:
# model test
voting_clf.score(x_test,y_test)

0.9677142857142857

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [17]:
# 각 분류 모델 학습
named_models = [
    ("RandomForest" , rnd_clf),
    ("ExtraTree" , etc_clf),
    ("LinearSVM" , svm_clf),
    ("MLP" , mlp_clf),
]
# model fitting
# 한 번에 훈련
models = [rnd_clf,etc_clf,svm_clf,mlp_clf]

for model in models:
    print("훈련 :",model)
    model.fit(X_train,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('RandomForest',
                              RandomForestClassifier(random_state=42)),
                             ('ExtraTree',
                              ExtraTreesClassifier(random_state=42)),
                             ('LinearSVM',
                              LinearSVC(max_iter=100, random_state=42)),
                             ('MLP', MLPClassifier(random_state=42))])

In [18]:
# 각 분류 모델의 성능 확인
[model.score for model in voting_clf.estimators_ ]

[<bound method ClassifierMixin.score of RandomForestClassifier(random_state=42)>,
 <bound method ClassifierMixin.score of ExtraTreesClassifier(random_state=42)>,
 <bound method ClassifierMixin.score of LinearSVC(max_iter=100, random_state=42)>,
 <bound method ClassifierMixin.score of MLPClassifier(random_state=42)>]

- Q. 어떤 모델의 성능이 가장 낮나요?
- A.

In [19]:
# 가장 성능이 낮은 모델 제거
del voting_clf.estimators_[2]

In [20]:
# model fitting
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('RandomForest',
                              RandomForestClassifier(random_state=42)),
                             ('ExtraTree',
                              ExtraTreesClassifier(random_state=42)),
                             ('LinearSVM',
                              LinearSVC(max_iter=100, random_state=42)),
                             ('MLP', MLPClassifier(random_state=42))])

In [21]:
# 모델 제거 후 성능 확인
voting_clf.score(x_test, y_test)

0.9677142857142857

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [10]:
estimators = [rnd_clf, etc_clf, svm_clf, mlp_clf]

In [22]:
X_val_predictions = np.empty((len(x_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(x_test)


####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [23]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_test)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다. 

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [24]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
X_test_predictions = np.empty((len(x_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(x_test)

In [25]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
y_pred = rnd_forest_blender.predict(X_test_predictions)


In [26]:
# model test
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.9818285714285714